In [196]:
import pandas as pd
import numpy as np
import pickle

In [197]:
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [198]:
import spotipy
import spotipy.util as util

import os
os.environ['SPOTIPY_CLIENT_ID'] = "6051b4896eda4feca0e0e28bed145907"
os.environ['SPOTIPY_CLIENT_SECRET'] = "1d7c94e540d741758cc103db6e028a96"
os.environ['SPOTIPY_REDIRECT_URI'] = "https://localhost:8080"

token = util.prompt_for_user_token("31zahq3gf5zm3oansaxx76hufvpm")

# AUTHENTICATION
sp = spotipy.Spotify(auth=token)


In [199]:
genre_prediction_model = pickle.load(open("model.dat", 'rb'))

In [200]:
key_map = {0: 'C',1: 'C#', 2: 'D',3: 'Eb',4: 'E',5: 'F',6: 'F#',7: 'G',8: 'Ab',9: 'A',10: 'Bb',11: 'B'}

In [201]:
time_map = {0: 'time_sign_1',1: 'time_sign_3',2: 'time_sign_4',3: 'time_sign_5'}

In [202]:
def get_key_dict(key):
    d = {}
    lst = np.eye(12).tolist()[key]
    for i in range(len(lst)):
        d[key_map[i]] = lst[i]
    return d


In [203]:
def transform_key_dict(feat):
    key_dict = get_key_dict(feat["key"])
    feat.update(key_dict)
    del feat['key']
    del feat['B']
    return feat


In [204]:
def get_time_dict(time):
    d = {}
    lst = np.eye(4).tolist()[time]
    for i in range(len(lst)):
        d[time_map[i]] = lst[i]
    return d

In [205]:
def transform_time_dict(feat):
    time_dict = get_time_dict(dict(map(reversed, time_map.items()))['time_sign_'+str(feat["time_signature"])])
    feat.update(time_dict)
    del feat['time_signature']
    return feat


In [206]:
def get_features(song_url):
    track = sp.track(song_url)
    features = sp.audio_features([track["id"]])[0]
    features.pop('type')
    features.pop('uri')
    features.pop('track_href')
    features.pop('analysis_url')
    features.pop('id')
    features = transform_key_dict(features)
    features = transform_time_dict(features)
    features['major'] = features['mode']
    del features['mode']
    return features


In [207]:
genre_map = {
    'ROCK': 0,
    'POP': 1,
    'LOFI': 2,
    'CLASSICAL': 3,
    'RAP': 4
}

## Prediction of genre of a track:

In [208]:
def predict_genre(song_url, model):
    data = pd.Series(get_features(song_url))
    predict_x = model.predict([data])
    inverted_dict = dict(map(reversed, genre_map.items()))
    return inverted_dict[predict_x[0]], sp.track(song_url)['artists'][0]['name'], sp.track(song_url)['name']


In [209]:
genre, artist, song = predict_genre("https://open.spotify.com/track/561jH07mF1jHuk7KlaeF0s?si=026c6b4b647448dc", genre_prediction_model)

In [210]:
print("Song:", song, "\nArtist:", artist, "\nGenre:", genre)

Song: Mockingbird 
Artist: Eminem 
Genre: RAP


In [211]:
genre, artist, song = predict_genre("https://open.spotify.com/track/1DfGPEHxTYeaJpiNA4xUb5?si=a139e57efe88451d", genre_prediction_model)

In [212]:
print("Song:", song, "\nArtist:", artist, "\nGenre:", genre)

Song: Für Elise 
Artist: Ludwig van Beethoven 
Genre: CLASSICAL


In [213]:
genre, artist, song = predict_genre("https://open.spotify.com/track/1u8c2t2Cy7UBoG4ArRcF5g?si=120021775ba341dd", genre_prediction_model)

In [214]:
print("Song:", song, "\nArtist:", artist, "\nGenre:", genre)

Song: Blank Space 
Artist: Taylor Swift 
Genre: POP


In [215]:
genre, artist, song = predict_genre("https://open.spotify.com/track/18URo35acNKRTHfEdjTcGn?si=08691573a3214586", genre_prediction_model)

In [216]:
print("Song:", song, "\nArtist:", artist, "\nGenre:", genre)

Song: Photograph 
Artist: Def Leppard 
Genre: ROCK


## Finding major genre the artist makes music in:

In [217]:
def artist_genre(artist_url, model):
    genre_frequency = [0,0,0,0,0]
    albums = sp.artist_albums(artist_url)["items"]
    artist = albums[0]['artists'][0]['name']
    for album in albums:
        tracks = sp.album_tracks(album["id"])["items"]
        for track in tracks:
            genre, x, y = predict_genre(track["id"], model)
            genre_frequency[genre_map[genre]] += 1
    return artist, dict(map(reversed, genre_map.items()))[genre_frequency.index(max(genre_frequency))]

In [218]:
artist, genre = artist_genre("https://open.spotify.com/artist/5JZ7CnR6gTvEMKX4g70Amv?si=dm7arc3QSICiTbUSFrgn2A", genre_prediction_model)
print('Artist:', artist, '\nGenre:', genre)

Artist: Lauv 
Genre: POP


In [219]:
artist, genre = artist_genre("https://open.spotify.com/artist/1iTPP9IvIq7CDNiSfjaSoh?si=1IymruUxSUukFDCDr34sHw", genre_prediction_model)
print('Artist:', artist, '\nGenre:', genre)

Artist: LofiClub 
Genre: LOFI
